# Introduction 

<a href="https://online.stanford.edu/courses/soe-ycsalgorithms1-algorithms-design-and-analysis-part-1">Stanford University Online - Algorithms: Design and Analysis, Part 1</a>
> "Perhaps the most important principle for the good algorithm designer is to refuse to be content." 
 -Aho, Hopcroft, and Ullman, _The Design and Analysis of Computer Algorithms_, 1974  
 
* [Integer Multiplication (Grade School)](#grade)
* [Karatsuba Multiplication (Integer Multiplication Redux)](#kara)
    * [A Recursive Algorithm](#rec)
    * [Optimization of Karatsuba Multiplication](#opt)
* [Course Topics](#course)
* [Merge Sort](#merge)
    * [The Sorting Problem](#prob)
    * [Pseudocode](#pseudo)
    * [Analysis](#anal)
* [Guiding Principles for Analysis of Algorithms](#princ)
* [Exploration Questions](#quest)

## Integer Multiplication (Grade School) <a class="anchor" id="grade"></a>

* __Primitive Operations__: add or multiply 2 single-digit numbers
* When breaking down problems, focus on the number of basic operations an algorithm performs as a function of the length of the input
* Integer Multiplication - 
    * __Input__:  2 n-digit numbers
    * __Output__: x * y
    * 5678 * 1234 = Sum of the product of 5678 and each of 1234 digits
        * 5678 * 4 = 22712
        * 5678 * 30 = 170340
        * 5678 * 200 = 1135600
        * 5678 * 1000 = 5678000
        * 22712 + 170340 + 1135600 + 5678000 = 7006652
    * Overall, the number of basic operations <= constant * n^2 (in this case, constant is 4)
        * 4 multiplications + 4 carries --> 8 basic operations --> 2n per partial product
        * 2n operations * 4 partial products = 2n ops * n partial products = 2n^2 basic operations 
        * Final addition of all partial products requires roughly the same number of operations (2n^2)
        * Total count is 4n^2 --> _quadratic_
    * Because the result is quadratic, if you double the number of digits (n), the number of operations will increase by a factor of 4, triple by a factor of 8, quadruple by a factor of 16 etc.

## Karatsuba Multiplication (Int Multip. Redux) <a class="anchor" id="kara"></a>

* Karatsuba Multiplication
    * __Input__: 2 n-digit numbers
    * __Output__: x * y
    * 5678 * 1234 =  
        * Step 0: Break into 2-digit numbers
            * a = 56
            * b = 78
            * c = 12
            * d = 34
        * Step 1: a * c = 56 * 12 = 672
        * Step 2: b * d = 78 * 34 = 2652
        * Step 3: (a+b) * (c + d) = (56 + 78) * (12 * 34) = 134 * 46 = 6164
        * Step 4: Subtract results from previous steps: 6164 - 2652 - 672 = 2840
        * Combine:
            * Pad Step 1 with 4 0's = 6720000
            * Don't pad Step 2 at all = 2652
            * Pad Step  4 with 2 0s: 28400
            * Sum: 7006652
* The main point is that there is more than a single way to solve the problem of integer multiplication, and some may be better than others

### A Recursive Algorithm <a class="anchor" id="rec"></a>

* Write x = 10^(n/2)a + b and y = 10(n/2)c + d, where abcd are n/2 digit integers
    * In general, any number can be represented this way, with each part a and b comprising of each half of the number (visually, not arithmetically), padded appropriately.
    * See Step 0 above
* x * y = (10^(n/2) a + b)(10^(n/2) c + d)
* Expanded for computation = (10^n)ac + (10^n/2)(ad + bc) + bd = K
    * This expression will now be called K for future reference
    * K assumes that n is an even integer, but for odds it's basically the same (n = 9 --> First 5 and then second 4 in the number)
    * K ignores the simple base case
    * K achieves the same result as 5678 * 1234 but deals only with _smaller numbers_
* Recursively compute __ac, ad, bc, bd__, pad with 0s, then add each part of K together
    * Base case would be 2 numbers that are sufficiently small (ie: 1 digit) are just multiplied and returned

### Optimization of Karatsuba Multiplication <a class="anchor" id="opt"></a>

* K --> x * y = (10^n)ac + (10^n/2)(ad + bc) + bd
    * We actually only care about 3 values (ac, bd, and (ad + bc)), because we don't really need ad and bc individually, just their sum. ___Can we compute this with only 3 recursive calls then?___
    * Step 1: recursively compute ac
    * Step 2: recursively compute bd
    * Step 3: recursively compute (a+c)(c+d)
        * Expanded: ac + ad + bc + bd
    * Step 4 - Gauss's Trick: Look at step 3 and subtract from it what we have calculated from Steps 1 and 2
        * Step 1 cancels out ac, Step 2 cancels out bd, leaving us with just (ad + bc)
    * Results in the same value (and really the same method) as the recursive call, but reduces the recursion at the expense of including an addition and a subtraction
    > Karatsuba Multiplication is an example of DIVIDE AND CONQUER

## Course Topics <a class="anchor" id="course"></a>

* Vocabulary for design and analysis of algorithms
    * Big O - ignore lower level operations and focus only on how time requirements scale with inputs
* Key algorithm design techniques 
    * Divide and Conquer - Break the problem into smaller parts that can be solved recursively, then somehow combine all of those into one result
        * Applies to integer multiplication, sorting, matrix multiplication, closest pair
        * Considered a "Master Method/Theorem" for general analysis
* Randomization in Algorithm Design
    * QuickSort, primality testing, graph partitioning, hashing
    * Different executions even when run on the same input
* Primitives for reasoning about graphs
    * Connectivity information, shortest paths, structure of information and social networks
    * Computation is so fast that it is essentially "free"
    * Very helpful for preprocessing
* Use and implementation of data structures
    * Arrays, vectors, lists, stacks, queues, heaps, balanced binary search trees, hashing and some variants, graphs
    * Graph - data structure that has vertices connected by "edges" and are great for modeling networks. Even though they are more complicated than arrays, there are still very fast primitives
    * Balanced Binary Search Trees  dynamically maintain an ordering on a set of elements while supporting a large number of queries that run in time logarithmic in the size of the set
    * Hash tables/Hash Maps - keep track of a dynamic set while supporting extremely fast insert and lookup queries

## Merge Sort <a class="anchor" id="merge"></a>

* Good introduction to divide and conquer
    * Improves over Selection, Insertion, Bubble sorts 
    * Above sorts have a quadratic dependence on input, merge sort has less
* Recursion-Tree method - "Master Method"

### The Sorting Problem <a class="anchor" id="sort"></a>

* __input__: unsorted array of numbers
* __output__: same numbers, sorted
* General Idea - recursively calls itself to the "lowest level" then goes back up 
* Imagine an array of 8 elements: 5 4 1 8 7 2 6 3
    * Splits in half recursively eventually returns:
        * 5418 --> 1 4 5 8
        * 7263 --> 2 3 6 7
    * Merge: combine the returned values to output the sorted list

### Pseudocode <a class="anchor" id="pseudo"></a>

* Ignores odd length lists, additional check for end of each arrays
1. Recursively sort 1st half of input array
    * Base Case: array with single element (or empty) is returned with no recursion
2. Recursively sort 2nd half of input array
    * Base Case: array with single element (or empty) is returned with no recursion
3. Merge two sorted sublists into 1 
    * C = output array (length = n)
    * A, B = 1st and 2nd sorted array (length = n/2)
    * i, j = iterators for traversing A and B respectively. Initialized at 1.
    * k = iterator for traversing C. Initialized at 0. --> BASICALLY A POINTER
    * For k = 1 to n:
        * if A(i) < B(j): C(k) = A(i) i++
        * else B(j) < A(i): C(k) = B(j) j ++
* Running Time - Imagine running the algorithm through a debugger, and determine how many times it performs an operation before terminating
* Merge Step <= 6n
    * Initialization Step (1 operation each = 2 Total)
    * For Loop: Comparison, Assignment, Increment i/j, Increment K (4 total per iteration of the loop)
    * Total = 2 initializations + (4 operations * n loops) = 4m + 2
    * Can make it even simpler to 6n (n >= 1)
* The small differences in how you might count the number of basic operations don't matter much

### Analysis <a class="anchor" id="anal"></a>

* __CLAIM__: Merge sort requires <= 6nlog(n) + 6n operations. 
    * Comparing to the quadratic dependency above, this is _much_ faster
* log base2 of n - Number of times you can divide n by 2 until the number drops below 1. 
    * log2(32) ---> 2 * 2 * 2 * 2 * 2 = 32 --> 5
    * Much smaller than whatever the input is 
* Recursion Tree Method - Write out all the work done by recursive merge sort algorithm in a tree structure, with the children of a given node corresponding to the recursive calls made by that node

<img src="resources/rec_tree.PNG">

* __At a given level j of this recursion, how many distinct sub-problems are there?__ 2^j
* __For each of these subproblems at level j, what is the input size?__ n/2^i
* When counting levels of recursion tree, it does not include work which will get done _in the recursion_
    * Number of problems at a given level j <= 2^j
    * Amount of workdone per sub-problem = 6(n/2^j)
        * Each input after it recursively = n/2^j
        * Merge subroutine is at most 6n
    * Total # of operations at level j <= 2^j +6(n/2^j) ==> 2^j cancels = 6n, independent of level j
        * simply put: number of problems per level increases by a factor of 2, but the work per problem decreases by a factor of 2
    * __Total Run Time of Merge Sort__ = number of levels  * upper bound
        * We know that there will be logn + 1 levels 
        * we just determined upper bound independent of level = 6n
        * 6n(logn + 1) ==> 6nlogn + 6n


## Guiding Principles for Analysis of Algorithms <a class="anchor" id="princ"></a>

1. Use worst-case analysis - find the _upper bound_ of run time
    * Particularly appropriate for "general purpose" routines
    * As opposed to average case analysis or any sort of pre-supposed benchmark inputs, which both require domain knowledge of the specific problem you are analyzing
    * BONUS: worst case is usually easier to analyze
2. In general, ignore constant factors and lower order terms
    * Mathematically easier
    * Constants ultimately rely on machine dependent factors like architecture/compiler and the programmer anyway
    * Lose very little predictive power
    * For crucial programs, constant factors need to be optimized. For understanding an analysis at this level, it's just unnecessary.
3. Asymptotic Analysis - Focus on rate of growth at large input size (ie: approaching infinity)
    * EG: Merge sort (6nlogn + 6n) "better than" other sort methods, like an insertion sort (1/2 n^2) IF AND ONLY IF n is sufficiently large. 
    * Modern computers solve a typical n instantaneously, so the only times an algorithmic approach really matters is at large input size
    
<img src="resources/sort_speed_graph.PNG">
<img src="resources/sort_speed_graph2.PNG">

> For the purposes of this course, a fast algorithm is one in which the worst-case running time grows slowly with input size. Always aspire toward linear time

## Exploration Questions <a class="anchor" id="quest"></a>

* How would the analysis of a merge sort change if there were duplicate numbers in the array?